# Import of required libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import plotly.graph_objs as go
import plotly.express as px
import dash
from dash import html
from dash import dcc

# Creation of dataframes

## Reading

In [2]:
covid_data = 'Data/covid-data.csv'
education_GDP_data = 'Data/Education-GDP.csv'
GDP_data = 'Data/GDP.csv'
gini_data = 'Data/gini.csv'
poverty_data = 'Data/poverty-explorer.csv'
trade_data = 'Data/trade.csv'
general_economy_data = 'Data/data_econo_covid.csv'

In [4]:
df_covid = pd.read_csv(covid_data, delimiter=',') #---> falta analizar
df_education = pd.read_csv(education_GDP_data, delimiter=',')
df_GDP = pd.read_csv(GDP_data, delimiter=',')
df_inequality = pd.read_csv(gini_data, delimiter=',')
df_poverty = pd.read_csv(poverty_data, delimiter=',')
df_trade = pd.read_csv(trade_data, delimiter=',')
df_economy = pd.read_csv(general_economy_data, delimiter=';') # ---> falta analizar

In [5]:
df_economy.head()

,iso_code,location,date,total_cases,total_deaths,stringency_index,population,gdp_per_capita,human_development_index,TC,TD,STI,POP,GDPCAP
0,AFG,Afghanistan,2019-12-31,0.0,0.0,0,38928341,"1803,987","0,498",#¡NUM!,#¡NUM!,#¡NUM!,"17,4772331","7,497754494"
1,AFG,Afghanistan,2020-01-01,0.0,0.0,0,38928341,"1803,987","0,498",#¡NUM!,#¡NUM!,#¡NUM!,"17,4772331","7,497754494"
2,AFG,Afghanistan,2020-01-02,0.0,0.0,0,38928341,"1803,987","0,498",#¡NUM!,#¡NUM!,#¡NUM!,"17,4772331","7,497754494"
3,AFG,Afghanistan,2020-01-03,0.0,0.0,0,38928341,"1803,987","0,498",#¡NUM!,#¡NUM!,#¡NUM!,"17,4772331","7,497754494"
4,AFG,Afghanistan,2020-01-04,0.0,0.0,0,38928341,"1803,987","0,498",#¡NUM!,#¡NUM!,#¡NUM!,"17,4772331","7,497754494"


## Filtering by area

In [11]:
# We don't consider the islands

south_america = ['Chile', 'Argentina','Uruguay','Paraguay','Bolivia','Peru','Brazil','Ecuador',
                'Venezuela', 'Colombia','Guyana','Suriname','French Guiana']
north_america = ['Canada','United States', 'Mexico','Guatemala','Honduras','Nicaragua','Costa Rica','Panama',
                'Dominican Republic', 'Jamaica','Cuba','Bahamas','Belize']

In [12]:
americas = south_america + north_america
df_americas = df_GDP[df_GDP['Entity'].isin(americas)]

In [29]:
df_poverty_renamed = df_poverty.rename(columns={'Country': 'Entity'})
df_americas = pd.merge(df_americas, df_education, on=['Entity', 'Year'], how='left')
df_americas = pd.merge(df_americas, df_inequality, on=['Entity', 'Year'], how='left')
df_americas = pd.merge(df_americas, df_trade, on=['Entity', 'Year'], how='left')
df_americas = pd.merge(df_americas, df_poverty_renamed, on=['Entity', 'Year'], how='left')

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: Passing 'suffixes' which cause duplicate columns {'Code_x'} in the result is deprecated and will raise a MergeError in a future version.
  after removing the cwd from sys.path.


### Modification of format dataframe df_covid

In [37]:
df_covid.head()

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,population,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million,Year
0,AFG,Asia,Afghanistan,2020-01-03,1.0,1.0,NaN,NaN,0.0,NaN,...,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN,2020
1,AFG,Asia,Afghanistan,2020-01-04,1.0,0.0,NaN,NaN,0.0,NaN,...,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN,2020
2,AFG,Asia,Afghanistan,2020-01-05,1.0,0.0,NaN,NaN,0.0,NaN,...,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN,2020
3,AFG,Asia,Afghanistan,2020-01-06,1.0,0.0,NaN,NaN,0.0,NaN,...,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN,2020
4,AFG,Asia,Afghanistan,2020-01-07,1.0,0.0,NaN,NaN,0.0,NaN,...,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN,2020


In [34]:
df_covid['date'] = pd.to_datetime(df_covid['date'])
df_covid['Year'] = df_covid['date'].dt.year
df_covid_grouped = df_covid.groupby(['Year', 'iso_code']).agg({'columna_a_agregar': 'sum'}).reset_index()

KeyError: "Column(s) ['columna_a_agregar'] do not exist"

# Analysis

In [40]:
descrtion = df_americas.describe()
descrtion

,Year,"Gross domestic product, constant prices - Percent change - Observations","Gross domestic product, constant prices - Percent change - Forecasts",Historical and more recent expenditure estimates,Gini coefficient,Trade (% of GDP),Share below $1 a day,Share below $2.15 a day,Share below $3.65 a day,Share below $6.85 a day,...,decile8_avg,decile9_avg,decile10_avg,bottom50_share,middle40_share,palma_ratio,s80_s20_ratio,p90_p10_ratio,p90_p50_ratio,p50_p10_ratio
count,1172.000000,1025.000000,171.000000,566.000000,446.000000,947.000000,448.000000,448.000000,448.000000,448.000000,...,446.000000,446.000000,446.000000,446.000000,446.000000,446.000000,446.000000,448.000000,448.000000,448.000000
mean,2003.923208,2.795708,3.741924,4.369025,0.478737,65.040680,2.530172,7.842076,16.362380,33.697450,...,27.196046,36.388259,75.591407,18.937941,44.159195,3.155789,14.693398,10.554556,3.073070,3.301527
std,14.110794,4.765327,7.596325,1.517905,0.074141,36.062448,3.276333,8.024619,13.922214,22.068158,...,23.897656,29.669577,49.173691,4.158512,2.679490,1.281379,7.726957,6.591006,0.585839,1.454625
min,1980.000000,-23.508000,-29.995000,0.773950,0.310191,11.545672,0.000000,0.055850,0.243559,0.490426,...,4.269791,6.199108,14.182703,10.766960,33.448048,1.126064,5.144568,4.229095,1.989736,2.101792
25%,1992.000000,0.852000,2.007500,3.283885,0.431895,39.829510,0.426006,1.247654,4.303067,15.344973,...,12.797218,18.196164,43.298095,15.952846,42.156209,2.221983,9.303715,6.939714,2.623022,2.695179
50%,2004.000000,3.394000,2.984000,4.350055,0.486623,57.777030,1.146826,5.293391,14.033969,35.185437,...,16.818754,23.453449,59.302615,18.446770,44.351133,2.961822,13.364509,9.528306,3.074071,3.004121
75%,2016.000000,5.099000,3.800000,5.360971,0.533703,83.175322,3.427055,12.286835,25.648158,50.353935,...,29.168003,40.641939,86.689842,21.660355,46.482944,3.953968,17.674554,11.926422,3.431687,3.594270
max,2028.000000,43.480000,62.288000,8.561060,0.632951,274.973100,23.174310,49.702816,69.943450,87.514430,...,109.781170,141.232320,264.254100,28.712519,51.207460,7.540501,72.681730,96.666660,4.975169,23.999998


In [45]:
frequency_entity = df_americas['Entity'].value_counts()
print(frequency_entity)


Argentina             49
Bahamas               49
Uruguay               49
United States         49
Suriname              49
Peru                  49
Paraguay              49
Panama                49
Nicaragua             49
Mexico                49
Jamaica               49
Honduras              49
Guyana                49
Guatemala             49
Ecuador               49
Dominican Republic    49
Costa Rica            49
Colombia              49
Chile                 49
Canada                49
Brazil                49
Bolivia               49
Belize                49
Venezuela             45
Name: Entity, dtype: int64


In [47]:
fig = px.line(df_americas, x='Year', y='Gini coefficient', color='Entity', title='Evolution  of Gini Coefficient')
fig.show()
